<a href="https://colab.research.google.com/github/intanelaqsha/Grievances-Event/blob/main/5_1_26_Related_Grievances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A. SCENARIO C STEP 1**

calling all plot/mill mentioned in the grievances.

In [ ]:
import pandas as pd

df = pd.read_csv("Grievances-Grid view.csv", dtype=str).fillna("")

# -------------------------
# Step 1: Load & normalize
# -------------------------

def clean_array(x):
    if x.strip() == "":
        return []
    return list(dict.fromkeys([i for i in x.split(",") if i.strip() != ""]))

df["plots_array"] = df["PIOConcessions-v2"].apply(lambda s: clean_array(s.replace(" ", "")))
df["mills_array"] = df["Mills"].apply(lambda s: clean_array(s.replace(" ", "")))

# Source: reproduce DuckDB logic exactly
def split_source(src):
    if src.strip() == "":
        return []
    tmp = src.replace(", ", "<COMMA_SPACE>")
    parts = tmp.split(",")
    cleaned = [p.strip().replace("<COMMA_SPACE>", ", ") for p in parts if p.strip() != ""]
    return list(dict.fromkeys(cleaned))

df["source_array"] = df["Source"].apply(split_source)

df["issues_array"] = df["Issues"].apply(
    lambda s: list(dict.fromkeys([i.strip() for i in s.split(",") if i.strip() != ""]))
)

# -------------------------
# Step 2: Unnest arrays
# -------------------------

g_plots = df.explode("plots_array")[["ID", "plots_array"]].rename(columns={"plots_array": "plot"})
g_mills = df.explode("mills_array")[["ID", "mills_array"]].rename(columns={"mills_array": "mill"})
g_issues = df.explode("issues_array")[["ID", "issues_array"]].rename(columns={"issues_array": "issue"})

# -------------------------
# Step 3: issue category
# -------------------------

issue_category = pd.DataFrame([
    ('Biodiversity loss', 'ENV'), ('Deforestation', 'ENV'), ('Fires', 'ENV'),
    ('Illegal Infrastructure', 'ENV'), ('Infrastructure Damage', 'ENV'),
    ('Peatland Loss', 'ENV'), ('Riparian Issues', 'ENV'), ('Environmental Pollution', 'ENV'),
    ('Corruption', 'SOC'), ('Forced Labor and/or Child Labor', 'SOC'),
    ('Gender and Ethnic Disparities', 'SOC'), ('Human Rights Violation', 'SOC'),
    ('Indigenous Peoples Conflict', 'SOC'), ('Labor Rights Violations', 'SOC'),
    ('Land Dispute', 'SOC'), ('Land Grabbing', 'SOC'), ('Labor Disputes', 'SOC'),
    ('Limited Access to Services', 'SOC'), ('Violence and/or Coercion', 'SOC'),
    ('Wage Dispute', 'SOC')
], columns=["issue", "category"])

# -------------------------
# Step 4: generate group_key
# -------------------------

plot_issue = g_plots.merge(g_issues, on="ID", how="inner") \
    .merge(issue_category, on="issue", how="left")

plot_issue["group_key"] = plot_issue["plot"] + "_" + plot_issue["category"].str.lower()

mill_issue = g_mills.merge(g_issues, on="ID", how="inner") \
    .merge(issue_category, on="issue", how="left")

mill_issue["group_key"] = mill_issue["mill"] + "_" + mill_issue["category"].str.lower()

# -------------------------
# Step 5: union
# -------------------------

combined = pd.concat([
    plot_issue[["group_key", "ID"]],
    mill_issue[["group_key", "ID"]]
], ignore_index=True)

# -------------------------
# Step 6: aggregation
# -------------------------

def flat_unique(series):
    out = []
    for arr in series:
        if isinstance(arr, list):
            out.extend(arr)
    return list(dict.fromkeys(out))

result = (
    combined.merge(df, on="ID", how="left")
    .groupby("group_key")
    .agg({
        "ID": lambda x: sorted(set(x), key=lambda v: v),   # LIST(DISTINCT ID ORDER BY ID)
        "plots_array": flat_unique,
        "mills_array": flat_unique,
        "source_array": flat_unique,
        "issues_array": flat_unique,
    })
    .reset_index()
)

result.rename(columns={"group_key": "MHGID"}, inplace=True)

# -------------------------
# Step 7: format output
# -------------------------

def fmt(x):
    if isinstance(x, list):
        if len(x) == 0:
            return ""
        return ", ".join(str(i) for i in x)
    return x

for col in ["ID", "plots_array", "mills_array", "source_array", "issues_array"]:
    result[col] = result[col].apply(fmt)

result.rename(columns={
    "ID": "grievance_IDs",
    "plots_array": "plots",
    "mills_array": "mills",
    "source_array": "sources",
    "issues_array": "issues"
}, inplace=True)

# -------------------------
# NEW: grievance_count
# -------------------------
result["grievance_count"] = result["grievance_IDs"].apply(
    lambda s: 0 if s == "" else len([i.strip() for i in s.split(",")])
)

result.to_csv("MHG_Newest_fix.csv", index=False)



# **B. GROUPING PLOT BY NAME SIMILARITY AND AREA PROXIMITY**

- if name similar and under proximity, grouping

In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.5 MB/s eta 0:00:00


In [ ]:
# ======================================================
# COMBINED STEP 1 & 2: NAME GROUPING + PROXIMITY REFINE
# ======================================================
import pandas as pd
import geopandas as gpd
from rapidfuzz import fuzz
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
import re

# ======================================================
# CONFIG
# ======================================================
INPUT_FILE = "Concessions-v2-Grid view.csv"
GPKG_FILE = "plots_v2_20251209.gpkg"
OUTPUT_FILE = "Concessions-with-group-proximity.csv"

FUZZY_THRESHOLD = 90
MAX_DIST_KM = 50

name_col = "Name"
id_col = "ID"

# ======================================================
# HELPER FUNCTIONS
# ======================================================

def clean_id(x):
    if pd.isna(x):
        return "UNKNOWN_ID"
    try:
        return str(int(float(x)))
    except:
        return str(x).strip()

def normalize_name(x):
    if pd.isna(x):
        return "UNKNOWN"
    s = str(x).strip()
    if s == "":
        return "UNKNOWN"
    if s.lower() == "unknown":
        return "UNKNOWN"
    if s.lower().startswith("no name"):
        return "NO_NAME"
    return s

def word_difference(a, b):
    words_a = a.split()
    words_b = b.split()
    return len(set(words_a) ^ set(words_b))

def extract_base_name(name):
    """
    Extract base name without trailing numbers
    'Palong 4' -> 'Palong'
    'Estate A 12' -> 'Estate A'
    'FELDA Bukit Jalor 1' -> 'FELDA Bukit Jalor'
    """
    # Remove trailing numbers (with optional separators)
    base = re.sub(r'\s+\d+$', '', name)
    return base.strip()

def dist_km(a, b):
    return a.distance(b) / 1000.0

# ======================================================
# STEP 1: LOAD & NAME GROUPING
# ======================================================
df = pd.read_csv(INPUT_FILE)

df[id_col] = df[id_col].apply(clean_id)
df[name_col] = df[name_col].apply(normalize_name)

# Extract base names for comparison
df["base_name"] = df[name_col].apply(extract_base_name)

# Grouping by name
groups = []
group_ids = []

for idx, row in df.iterrows():
    name = row[name_col]
    base_name = row["base_name"]
    pid = row[id_col]

    # RULE 1: UNKNOWN or NO NAME → own group
    if name == "UNKNOWN" or name == "NO_NAME":
        group_ids.append(f"{pid}G")
        continue

    # RULE 2: PT Perkebunan Nusantara → exact match only
    if name.lower().startswith("pt perkebunan nusantara"):
        assigned = False
        for g in groups:
            if g["name"].lower().startswith("pt perkebunan nusantara"):
                if word_difference(name, g["name"]) == 0:
                    group_ids.append(f"{g['root_id']}G")
                    assigned = True
                    break
        if not assigned:
            groups.append({"name": name, "base_name": base_name, "root_id": pid})
            group_ids.append(f"{pid}G")
        continue

    # RULE 3: CHECK BASE NAME FIRST (for "Name + Number" pattern)
    assigned = False

    for g in groups:
        if g["name"] in ["UNKNOWN", "NO_NAME"]:
            continue
        if g["name"].lower().startswith("pt perkebunan nusantara"):
            continue

        # Check if base names match exactly
        if base_name == g["base_name"] and base_name != name:
            # Same base name (e.g., both "Palong") → group together!
            group_ids.append(f"{g['root_id']}G")
            assigned = True
            break

    if assigned:
        continue

    # RULE 4: NORMAL FUZZY GROUPING (if base name didn't match)
    for g in groups:
        if g["name"] in ["UNKNOWN", "NO_NAME"]:
            continue
        if g["name"].lower().startswith("pt perkebunan nusantara"):
            continue

        sim = fuzz.token_sort_ratio(name, g["name"])
        if sim >= FUZZY_THRESHOLD:
            group_ids.append(f"{g['root_id']}G")
            assigned = True
            break

    if not assigned:
        groups.append({"name": name, "base_name": base_name, "root_id": pid})
        group_ids.append(f"{pid}G")

df["GroupID"] = group_ids

print(f"Step 1 complete: {len(groups)} name-based groups created")

# Show Palong grouping
palong_df = df[df["base_name"].str.contains("Palong", na=False, case=False)]
if len(palong_df) > 0:
    print("\nPalong plots grouping:")
    print(palong_df[["ID", "Name", "base_name", "GroupID"]])

# ======================================================
# STEP 2: LOAD GEOMETRY & PROXIMITY REFINE
# ======================================================
gdf = gpd.read_file(GPKG_FILE)
gdf[id_col] = gdf[id_col].astype(str)

# Merge geometry
merged = df.merge(gdf[[id_col, "geometry"]], on=id_col, how="left")
geo = gpd.GeoDataFrame(merged, geometry="geometry")

# Convert CRS & calculate centroids
geo = geo.to_crs(3857)
geo["centroid"] = geo.geometry.centroid

# ======================================================
# PROXIMITY REFINE (ONLY FOR ELIGIBLE GROUPS)
# ======================================================
refined_group = []

for group_id, sub in geo.groupby("GroupID"):

    # Skip solo plots
    if len(sub) == 1:
        refined_group.append((sub.index[0], group_id))
        continue

    # Check if UNKNOWN/NO_NAME group
    names = sub[name_col].unique()
    if len(names) == 1 and names[0] in ["UNKNOWN", "NO_NAME"]:
        for idx, pid in zip(sub.index, sub[id_col]):
            refined_group.append((idx, f"{pid}G"))
        continue

    # Normal proximity refinement
    ids = sub[id_col].tolist()
    cents = sub["centroid"].tolist()
    indices = sub.index.tolist()
    n = len(ids)

    # Build adjacency matrix
    adj = []
    for i in range(n):
        row = []
        for j in range(n):
            if i == j:
                row.append(1)
            else:
                d = dist_km(cents[i], cents[j])
                row.append(1 if d <= MAX_DIST_KM else 0)
        adj.append(row)

    # Find connected components
    graph = csr_matrix(adj)
    n_components, labels = connected_components(graph, directed=False)

    if n_components == 1:
        # All connected → keep original GroupID
        for idx in indices:
            refined_group.append((idx, group_id))
    else:
        # Split into sub-groups
        for comp_id in range(n_components):
            comp_indices = [indices[i] for i in range(n) if labels[i] == comp_id]

            if len(comp_indices) == 1:
                idx = comp_indices[0]
                pid = geo.loc[idx, id_col]
                refined_group.append((idx, f"{pid}G"))
            else:
                first_id = geo.loc[comp_indices[0], id_col]
                new_group = f"{first_id}G"
                for idx in comp_indices:
                    refined_group.append((idx, new_group))

# Assign final groups
idx_map = {i: g for i, g in refined_group}
geo["GroupID_proximity"] = geo.index.map(idx_map)

print(f"\nStep 2 complete: Proximity refinement done")

# ======================================================
# STATISTICS & SAVE
# ======================================================
result = geo.drop(columns=["geometry", "centroid"])

# Stats
group_sizes = result.groupby("GroupID_proximity").size()
print(f"\nFinal statistics:")
print(f"  Total plots: {len(result)}")
print(f"  Solo plots: {(group_sizes == 1).sum()}")
print(f"  Groups with 2+ plots: {(group_sizes > 1).sum()}")
print(f"  Largest group: {group_sizes.max()} plots")

# Show Palong final result
palong_result = result[result["base_name"].str.contains("Palong", na=False, case=False)]
if len(palong_result) > 0:
   # print("\nPalong plots FINAL:")
    print(palong_result[["ID", "Name", "GroupID", "GroupID_proximity"]])

# Save
result.to_csv(OUTPUT_FILE, index=False)
print(f"\nSaved: {OUTPUT_FILE}")

Step 1 complete: 7120 name-based groups created

Palong plots grouping:
         ID                                               Name  \
4608   9033                        FGVPM Palong Timur 5 Estate   
4624   9049                                      Palong Estate   
5711  10136                                     Palong Timur 3   
5712  10137                        FGVPM Palong Timur 6 Estate   
5713  10138                                     Palong Timur 1   
5715  10140                                     Palong Timur 2   
5726  10151                                           Palong 3   
5948  10375                                           Palong 4   
5949  10376                                           Palong 6   
5950  10377                                           Palong 2   
5951  10378                                           Palong 1   
5953  10380  Lee Brothers Plantation & Realty (M) Sdn Bhd (...   
5964  10391                                    FGVPM Palong 17   
5965

/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiPolygon' is converted to 'MultiPolygon Z'
  return ogr_read(



Step 2 complete: Proximity refinement done

Final statistics:
  Total plots: 10759
  Solo plots: 7836
  Groups with 2+ plots: 958
  Largest group: 54 plots
         ID                                               Name GroupID  \
4608   9033                        FGVPM Palong Timur 5 Estate   9033G   
4624   9049                                      Palong Estate   9049G   
5711  10136                                     Palong Timur 3  10136G   
5712  10137                        FGVPM Palong Timur 6 Estate   9033G   
5713  10138                                     Palong Timur 1  10136G   
5715  10140                                     Palong Timur 2  10136G   
5726  10151                                           Palong 3  10151G   
5948  10375                                           Palong 4  10151G   
5949  10376                                           Palong 6  10151G   
5950  10377                                           Palong 2  10151G   
5951  10378                  

# **C. COMBINE GROUP NAME AND PLOT/MILL GRIEVANCES**

-lookup group name

# FINAL MERGED WITH PLOT GROUP AND MILL GROUP LOOKUP.

In [ ]:
import pandas as pd
import re

# ============================
# Load Files
# ============================
mhg = pd.read_csv("MHG_Newest_fix.csv", dtype=str).fillna("")
plot = pd.read_csv("Concessions-with-group-proximity.csv", dtype=str).fillna("")
mill = pd.read_csv("Mills-Grid view.csv", dtype=str).fillna("")

# ============================
# Helper: Ambil angka sebelum "_"
# ============================
def extract_id(x):
    if pd.isna(x):
        return ""
    # PO100… adalah Mill ID
    if x.startswith("PO100"):
        m = re.match(r"(PO\d+)_", x)
        if m:
            return m.group(1)
        return x
    # Angka biasa adalah Plot ID
    m = re.match(r"(\d+)_", x)
    if m:
        return m.group(1)
    return x

mhg["BaseID"] = mhg["MHGID"].apply(extract_id)
mhg["is_mill"] = mhg["MHGID"].apply(lambda x: x.startswith("PO100"))

# ============================
# Build mapping from PLOT file
# ============================
plot["ID_clean"] = plot["ID"].apply(
    lambda x: re.match(r"(\d+)", x).group(1) if re.match(r"(\d+)", x) else x
)

plot_group_map = plot.set_index("ID_clean")["GroupID_proximity"].to_dict()
plot_group_name_map = plot.set_index("ID_clean")["Group Name"].to_dict()
plot_group_airtable_map = plot.set_index("ID_clean")["GroupAirtableRecID"].to_dict()
plot_name_map = plot.set_index("ID_clean")["Name"].to_dict()

# Build reverse mapping: GroupID_proximity → list of all plot IDs in that group
group_to_plots = {}
for idx, row in plot.iterrows():
    grp = row["GroupID_proximity"]
    pid = row["ID_clean"]
    if grp not in group_to_plots:
        group_to_plots[grp] = []
    group_to_plots[grp].append(pid)

# ============================
# Build mapping from MILL file
# ============================
mill["UML_ID_clean"] = mill["UML_ID"].apply(
    lambda x: re.match(r"(PO\d+)", x).group(1) if re.match(r"(PO\d+)", x) else x
)

mill_group_name_map = mill.set_index("UML_ID_clean")["Group_Name"].to_dict()
mill_group_airtable_map = mill.set_index("UML_ID_clean")["GroupAirtableRecID"].to_dict()
mill_name_map = mill.set_index("UML_ID_clean")["Mill_Name"].to_dict()

# ============================
# Determine GroupID_proximity per base ID
# ============================
def get_group_id(row):
    if row["is_mill"]:
        return ""  # Mills don't have GroupID_proximity
    else:
        return plot_group_map.get(row["BaseID"], "")

mhg["GroupID_proximity"] = mhg.apply(get_group_id, axis=1)

# ============================
# Issue group from suffix
# ============================
mhg["issue_group"] = mhg["MHGID"].apply(
    lambda x: x.split("_")[1] if "_" in x else ""
)

# ============================
# Build grouping dict
# ============================
group_dict = {}

for idx, row in mhg.iterrows():
    base = row["BaseID"]
    grp = row["GroupID_proximity"]
    issue = row["issue_group"]
    is_mill = row["is_mill"]

    if is_mill:
        # Mills: No merge with others, each is unique
        group_key = f"MILL-{row['MHGID']}"
    elif grp == "":
        # No group found
        group_key = f"{base}-{issue}"
    else:
        # Normal plot grouping
        group_key = f"{grp}-{issue}"

    group_dict.setdefault(group_key, []).append(idx)

# ============================
# Merge Process
# ============================
merged_rows = []
merge_count = 0

cols_to_merge = [
    c for c in mhg.columns
    if c not in ["MHGID", "BaseID", "GroupID_proximity", "issue_group", "is_mill"]
]

for group_key, indices in group_dict.items():

    # only 1 → no merge
    if len(indices) == 1:
        row = mhg.loc[indices[0]].copy()
        base = row["BaseID"]
        is_mill = row["is_mill"]

        if is_mill:
            # Mill lookup
            row["Name"] = mill_name_map.get(base, "")
            row["Mill_Name"] = mill_name_map.get(base, "")
            row["Group Name"] = mill_group_name_map.get(base, "")
            row["GroupAirtableRecID"] = mill_group_airtable_map.get(base, "")
            row["Plot_group"] = ""  # Mills don't have plot groups
        else:
            # Plot lookup
            row["Name"] = plot_name_map.get(base, "")
            row["Mill_Name"] = ""  # Plots don't have mill names
            row["Group Name"] = plot_group_name_map.get(base, "")
            row["GroupAirtableRecID"] = plot_group_airtable_map.get(base, "")

            # Expand plot_group: get ALL plots in this GroupID_proximity
            grp = row["GroupID_proximity"]
            if grp and grp in group_to_plots:
                row["Plot_group"] = ", ".join(sorted(group_to_plots[grp]))
            else:
                row["Plot_group"] = base

        merged_rows.append(row.to_dict())
        continue

    # merge happens
    merge_count += (len(indices) - 1)
    df = mhg.loc[indices]

    rep = df.iloc[0].copy()
    rep_base = rep["BaseID"]
    is_mill = rep["is_mill"]

    # merge unique values
    for col in cols_to_merge:
        vals = df[col].tolist()
        uniq = sorted(set(sum([
            v.split(", ") if ", " in v else [v] for v in vals
        ], [])))
        uniq = [v for v in uniq if v != ""]
        rep[col] = ", ".join(uniq)

    if is_mill:
        # Mill lookup
        rep["Name"] = mill_name_map.get(rep_base, "")
        rep["Mill_Name"] = mill_name_map.get(rep_base, "")
        rep["Group Name"] = mill_group_name_map.get(rep_base, "")
        rep["GroupAirtableRecID"] = mill_group_airtable_map.get(rep_base, "")
        rep["Plot_group"] = ""  # Mills don't have plot groups
    else:
        # Plot lookup
        rep["Name"] = plot_name_map.get(rep_base, "")
        rep["Mill_Name"] = ""  # Plots don't have mill names
        rep["Group Name"] = plot_group_name_map.get(rep_base, "")
        rep["GroupAirtableRecID"] = plot_group_airtable_map.get(rep_base, "")

        # Expand plot_group: get ALL plots in this GroupID_proximity
        grp = rep["GroupID_proximity"]
        if grp and grp in group_to_plots:
            rep["Plot_group"] = ", ".join(sorted(group_to_plots[grp]))
        else:
            # Fallback: use BaseIDs from merged rows
            plot_ids = sorted(set(df["BaseID"].tolist()))
            rep["Plot_group"] = ", ".join(plot_ids)

    # recalc grievance_count from updated grievance_IDs
    if "grievance_IDs" in rep:
        gid_list = rep["grievance_IDs"].split(", ")
        gid_list = [x for x in gid_list if x != ""]
        rep["grievance_count"] = str(len(set(gid_list)))

    merged_rows.append(rep.to_dict())

# ============================
# Build final DF
# ============================
result = pd.DataFrame(merged_rows)

# Drop helper columns from output
result = result.drop(columns=["BaseID", "issue_group", "is_mill"], errors="ignore")

# Reorder columns untuk clarity (Name dan Mill_Name di depan)
cols_order = ["MHGID", "Name", "Mill_Name", "GroupID_proximity", "Plot_group",
              "Group Name", "GroupAirtableRecID"]
other_cols = [c for c in result.columns if c not in cols_order]
result = result[cols_order + other_cols]

print("Jumlah sebelum merge :", len(mhg))
print("Jumlah sesudah merge :", len(result))
print("Total rows merged    :", merge_count)

# Show sample of mills
mill_results = result[result["MHGID"].str.startswith("PO100", na=False)]
if len(mill_results) > 0:
    print("\nSample Mill results:")
    print(mill_results[["MHGID", "Name", "Mill_Name", "Group Name"]].head())

# Show sample of plots
plot_results = result[~result["MHGID"].str.startswith("PO100", na=False)]
if len(plot_results) > 0:
    print("\nSample Plot results:")
    print(plot_results[["MHGID", "Name", "Plot_group", "Group Name"]].head())

# ============================
# Export
# ============================
result.to_csv("MHG_Merged_Final.csv", index=False)
print("\nFile saved as MHG_Merged_Final.csv")

Jumlah sebelum merge : 1291
Jumlah sesudah merge : 1051
Total rows merged    : 240

Sample Mill results:
                MHGID           Name      Mill_Name       Group Name
714  PO1000000017_soc  AGROPALMA/CPA  AGROPALMA/CPA        AGROPALMA
715  PO1000000099_env      PERLABIAN      PERLABIAN            SIPEF
716  PO1000000099_soc      PERLABIAN      PERLABIAN            SIPEF
717  PO1000000109_env   PAMOL KLUANG   PAMOL KLUANG  IOI CORPORATION
718  PO1000000109_soc   PAMOL KLUANG   PAMOL KLUANG  IOI CORPORATION

Sample Plot results:
       MHGID                         Name           Plot_group  \
0  10060_env            LADANG SSI SG ARA                10060   
1  10136_env               Palong Timur 3  10136, 10138, 10140   
2  10137_env  FGVPM Palong Timur 6 Estate          10137, 9033   
3  10289_env        Ladang  Sungai Pinang                10289   
4  10681_env    Puncak Niaga Holdings Bhd                10681   

                Group Name  
0          YPJ PLANTATIONS  
1   

# D. GROUPING BY THE GROUP NAME

if the group name is same, merge. others/unknown, dont merged. (mills use proximity)

In [ ]:
import pandas as pd
import geopandas as gpd
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from shapely.geometry import Point

# ======================================================
# CONFIG
# ======================================================
INPUT_FILE = "MHG_Merged_Final.csv"
GPKG_FILE = "plots_v2_20251209.gpkg"
MILLS_FILE = "Mills-Grid view.csv"  # File mills dengan kolom GPS/Latitude/Longitude
OUTPUT_FILE = "MHG_Final_GroupName_Proximity.csv"

MAX_DIST_KM = 50  # adjustable

# ======================================================
# LOAD DATA
# ======================================================
df = pd.read_csv(INPUT_FILE, dtype=str).fillna("")

# ======================================================
# ISSUE GROUP
# ======================================================
def get_issue(mhgid):
    if mhgid.endswith("_env"):
        return "env"
    if mhgid.endswith("_soc"):
        return "soc"
    return ""

df["issue_group"] = df["MHGID"].apply(get_issue)

# ======================================================
# MILL IDENTIFIER
# ======================================================
def is_mill(mhgid):
    return mhgid.startswith("PO100")

df["is_mill"] = df["MHGID"].apply(is_mill)

# ======================================================
# LOOKUP TABLES (PLOT & MILL NAME)
# ======================================================
mhgid_to_plot_name = df.set_index("MHGID")["Name"].to_dict()
mhgid_to_mill_name = df.set_index("MHGID")["Mill_Name"].to_dict()

# ======================================================
# LOAD GEOMETRY - PLOTS
# ======================================================
gdf_plots = gpd.read_file(GPKG_FILE)[["ID", "geometry"]]
gdf_plots["ID"] = gdf_plots["ID"].astype(str)

# ======================================================
# LOAD GEOMETRY - MILLS
# ======================================================
mills_df = pd.read_csv(MILLS_FILE, dtype=str).fillna("")

# Parse koordinat mills
def parse_mill_coords(row):
    """Extract lat, lon from mills data"""
    try:
        # Coba dari kolom Latitude, Longitude
        if row.get("Latitude") and row.get("Longitude"):
            lat = float(row["Latitude"])
            lon = float(row["Longitude"])
            return Point(lon, lat)

        # Coba dari kolom GPS (format: "lat, lon")
        if row.get("GPS"):
            coords = row["GPS"].split(",")
            if len(coords) == 2:
                lat = float(coords[0].strip())
                lon = float(coords[1].strip())
                return Point(lon, lat)
    except:
        pass
    return None

mills_df["geometry"] = mills_df.apply(parse_mill_coords, axis=1)
mills_df = mills_df[mills_df["geometry"].notna()]

# Buat GeoDataFrame untuk mills
gdf_mills = gpd.GeoDataFrame(
    mills_df,
    geometry="geometry",
    crs="EPSG:4326"
)

# Kolom ID mills (sesuaikan dengan nama kolom di file Mills)
# Asumsi ada kolom "Mill ID" atau "ID" atau "Name"
if "Mill ID" in gdf_mills.columns:
    gdf_mills["ID"] = gdf_mills["Mill ID"].astype(str)
elif "ID" in gdf_mills.columns:
    gdf_mills["ID"] = gdf_mills["ID"].astype(str)
elif "Name" in gdf_mills.columns:
    gdf_mills["ID"] = gdf_mills["Name"].astype(str)
elif "UML_ID" in gdf_mills.columns: # FIX: Add check for 'UML_ID'
    gdf_mills["ID"] = gdf_mills["UML_ID"].astype(str)

gdf_mills = gdf_mills[["ID", "geometry"]]

# ======================================================
# MERGE GEOMETRY KE MAIN DF
# ======================================================
df["BaseID"] = df["MHGID"].str.extract(r"^(PO\d+|\d+)")

# Merge plots
geo = df.merge(gdf_plots, left_on="BaseID", right_on="ID", how="left", suffixes=("", "_plot"))

# Merge mills (untuk mill rows)
geo = geo.merge(gdf_mills, left_on="BaseID", right_on="ID", how="left", suffixes=("", "_mill"))

# Combine geometry: gunakan geometry_mill jika ada, kalau tidak pakai geometry
geo["final_geometry"] = geo.apply(
    lambda r: r["geometry_mill"] if pd.notna(r.get("geometry_mill")) else r.get("geometry"),
    axis=1
)

geo = gpd.GeoDataFrame(geo, geometry="final_geometry", crs=gdf_plots.crs)

# Convert ke projected CRS untuk distance calculation
geo = geo.to_crs(3857)
geo["centroid"] = geo.geometry.centroid

# ======================================================
# DISTANCE FUNCTION
# ======================================================
def dist_km(a, b):
    return a.distance(b) / 1000.0

# ======================================================
# PROXIMITY FUNCTION (UNTUK PLOTS + MILLS)
# ======================================================
def split_by_proximity(entities_gdf, max_km):
    """
    Return: list of index-lists
    Each inner list = one proximity-based component
    Works for both plots and mills
    """
    cents = entities_gdf["centroid"].tolist()
    indices = entities_gdf.index.tolist()
    n = len(indices)

    if n <= 1:
        return [indices]

    adj = []
    for i in range(n):
        row = []
        for j in range(n):
            if i == j:
                row.append(1)
            else:
                row.append(1 if dist_km(cents[i], cents[j]) <= max_km else 0)
        adj.append(row)

    graph = csr_matrix(adj)
    n_components, labels = connected_components(graph, directed=False)
    n_components = int(n_components)

    components = []
    for comp in range(n_components):
        comp_indices = [indices[i] for i in range(n) if labels[i] == comp]
        components.append(comp_indices)

    return components

# ======================================================
# FINAL GROUPING
# ======================================================
final_rows = []
merge_count = 0

for issue, issue_df in geo.groupby("issue_group"):

    if issue == "":
        continue

    for group_id, sub in issue_df.groupby("GroupAirtableRecID"):

        # ==================================================
        # CASE A: NO GroupAirtableRecID → fallback
        # ==================================================
        if group_id == "":
            for _, r in sub.iterrows():
                name = r["Mill_Name"] if r["is_mill"] else r["Name"]
                r["MHGID"] = f"{name}_{issue}" if name else r["MHGID"]
                r["Group_MHGID"] = r["MHGID"]
                final_rows.append(r.to_dict())
            continue

        # ==================================================
        # CASE B: HAS GroupAirtableRecID
        # ==================================================
        # Pisahkan plots dan mills yang punya geometry
        plots = sub[~sub["is_mill"] & sub["centroid"].notna()]
        mills = sub[sub["is_mill"] & sub["centroid"].notna()]

        # Mills tanpa geometry (fallback)
        mills_no_geo = sub[sub["is_mill"] & sub["centroid"].isna()]

        # --------------------------------------------------
        # 0–1 entity (plot + mill) → no proximity split
        # --------------------------------------------------
        if len(plots) + len(mills) <= 1:
            rep = sub.iloc[0].copy()
            rep["MHGID"] = f"{rep['Group Name']}_{issue}"
            rep["Group_MHGID"] = ", ".join(sorted(sub["MHGID"].tolist()))

            # BUILD Name & Mill_Name FROM Group_MHGID
            plot_names, mill_names = [], []
            for mid in rep["Group_MHGID"].split(","):
                mid = mid.strip()
                if mid.startswith("PO10"):
                    n = mhgid_to_mill_name.get(mid, "")
                    if n:
                        mill_names.append(n)
                else:
                    n = mhgid_to_plot_name.get(mid, "")
                    if n:
                        plot_names.append(n)

            rep["Name"] = ", ".join(sorted(set(plot_names)))
            rep["Mill_Name"] = ", ".join(sorted(set(mill_names)))

            final_rows.append(rep.to_dict())
            continue

        # ==================================================
        # PROXIMITY SPLIT (PLOTS + MILLS COMBINED)
        # ==================================================
        # Gabungkan plots dan mills yang punya geometry
        entities_with_geo = pd.concat([plots, mills])

        if len(entities_with_geo) == 0:
            # Kalau semua tidak punya geometry, fallback
            rep = sub.iloc[0].copy()
            rep["MHGID"] = f"{rep['Group Name']}_{issue}"
            rep["Group_MHGID"] = ", ".join(sorted(sub["MHGID"].tolist()))

            plot_names, mill_names = [], []
            for mid in rep["Group_MHGID"].split(","):
                mid = mid.strip()
                if mid.startswith("PO10"):
                    n = mhgid_to_mill_name.get(mid, "")
                    if n:
                        mill_names.append(n)
                else:
                    n = mhgid_to_plot_name.get(mid, "")
                    if n:
                        plot_names.append(n)

            rep["Name"] = ", ".join(sorted(set(plot_names)))
            rep["Mill_Name"] = ", ".join(sorted(set(mill_names)))
            final_rows.append(rep.to_dict())
            continue

        # Split by proximity (plots + mills together)
        components = split_by_proximity(entities_with_geo, MAX_DIST_KM)

        for comp_indices in components:
            comp_df = geo.loc[comp_indices]

            # Tambahkan mills tanpa geometry ke semua components
            full_df = pd.concat([comp_df, mills_no_geo])

            # FIX: Check if full_df is empty before proceeding
            if full_df.empty:
                continue

            rep = full_df.iloc[0].copy()
            rep["MHGID"] = f"{rep['Group Name']}_{issue}"
            rep["Group_MHGID"] = ", ".join(sorted(full_df["MHGID"].tolist()))

            # BUILD Name & Mill_Name FROM Group_MHGID
            plot_names, mill_names = [], []
            for mid in rep["Group_MHGID"].split(","):
                mid = mid.strip()
                if mid.startswith("PO10"):
                    n = mhgid_to_mill_name.get(mid, "")
                    if n:
                        mill_names.append(n)
                else:
                    n = mhgid_to_plot_name.get(mid, "")
                    if n:
                        plot_names.append(n)

            rep["Name"] = ", ".join(sorted(set(plot_names)))
            rep["Mill_Name"] = ", ".join(sorted(set(mill_names)))

            # Merge list columns
            for col in ["grievance_IDs", "plots", "mills", "sources", "issues"]:
                vals = []
                for v in full_df[col]:
                    if v:
                        vals.extend([x.strip() for x in v.split(",")])
                rep[col] = ", ".join(sorted(set(vals)))

            # Plot_group (hanya dari plots)
            plots_in_comp = comp_df[~comp_df["is_mill"]]
            if len(plots_in_comp) > 0:
                rep["Plot_group"] = ", ".join(sorted(set(plots_in_comp["BaseID"].tolist())))
            else:
                rep["Plot_group"] = ""

            # grievance_count
            gids = [g.strip() for g in rep["grievance_IDs"].split(",") if g.strip()]
            rep["grievance_count"] = str(len(set(gids)))

            if len(full_df) > 1:
                merge_count += len(full_df) - 1

            final_rows.append(rep.to_dict())

# ======================================================
# FINALIZE
# ======================================================
final_df = pd.DataFrame(final_rows)
final_df = final_df.drop(
    columns=["issue_group", "is_mill", "centroid", "final_geometry",
             "geometry", "geometry_plot", "geometry_mill", "ID", "ID_plot", "ID_mill"],
    errors="ignore"
)

print("=" * 60)
print("FINAL GROUPING DONE (PLOTS + MILLS PROXIMITY)")
print("Merged rows :", merge_count)
print("Final rows  :", len(final_df))
print("=" * 60)

final_df.to_csv(OUTPUT_FILE, index=False)
print(f"Saved: {OUTPUT_FILE}")

/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiPolygon' is converted to 'MultiPolygon Z'
  return ogr_read(


FINAL GROUPING DONE (PLOTS + MILLS PROXIMITY)
Merged rows : 313
Final rows  : 738
Saved: MHG_Final_GroupName_Proximity.csv
